In [1]:
import keras ##can use import tensorflow.contrib.keras as keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout,BatchNormalization

Using TensorFlow backend.


In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
from PIL import Image
from random import shuffle,seed
from pprint import pprint
import os

In [3]:
batch_size = 22
image_size = (56,56)

src_dir = '/home/shashank/Desktop/Project_IITD/Painter-by-Numbers/final_train_data/'
dest_dir = '/home/shashank/Desktop/Project_IITD/Painter-by-Numbers/final_train_batches/'

In [4]:
files = []
for fx in os.listdir(src_dir):
    files.append(os.path.join(src_dir,fx))
pprint(files[:3])
seed(0)
shuffle(files)
pprint(files[:3])  

['/home/shashank/Desktop/Project_IITD/Painter-by-Numbers/final_train_data/31624.jpg',
 '/home/shashank/Desktop/Project_IITD/Painter-by-Numbers/final_train_data/1795.jpg',
 '/home/shashank/Desktop/Project_IITD/Painter-by-Numbers/final_train_data/1957.jpg']
['/home/shashank/Desktop/Project_IITD/Painter-by-Numbers/final_train_data/29512.jpg',
 '/home/shashank/Desktop/Project_IITD/Painter-by-Numbers/final_train_data/101209.jpg',
 '/home/shashank/Desktop/Project_IITD/Painter-by-Numbers/final_train_data/10576.jpg']


In [5]:
def load_image(fx):
    im = Image.open(fx)
    im = im.resize(image_size)
    im = np.array(im)
    im = im/255.0
    #print im.shape,':',fx
    im = np.reshape(im,(1,image_size[0],image_size[1],3))
    return im

In [6]:
df = pd.read_csv('/home/shashank/Desktop/Project_IITD/Painter-by-Numbers/final_train_labels/final_train_labels.csv')
df = df.drop('Unnamed: 0',axis=1)
df


,filename,artist,title
0,37256.jpg,Claude Monet,Gardener's House at Antibes
1,3016.jpg,Pierre-Auguste Renoir,Ambroise Vollard Portrait
2,28824.jpg,Claude Monet,Haystacks
3,16139.jpg,Claude Monet,Vetheuil
4,11233.jpg,Claude Monet,Rouen Cathedral
5,15562.jpg,Claude Monet,"Rouen Cathedral, evening, harmony in brown"
6,37160.jpg,Claude Monet,"Weeping Willow, Giverny"
7,25299.jpg,Camille Pissarro,The Hermitage at Pontoise
8,3603.jpg,Pierre-Auguste Renoir,By the Water (Near the Lake)
9,38252.jpg,Claude Monet,Trees in Bloom


In [7]:
artist_names = np.unique(df['artist'])
print artist_names
artist_names = artist_names.tolist()
print artist_names.index('Claude Monet')

['Boris Kustodiev' 'Camille Pissarro' 'Claude Monet' 'Ilya Repin'
 'Jean-Baptiste-Camille Corot' 'Martiros Saryan' 'Paul Gauguin'
 'Pierre-Auguste Renoir' 'Pyotr Konchalovsky']
2


In [8]:
o_h_base = np.zeros(len(artist_names))
print o_h_base.shape

(9,)


In [9]:
def load_labels(fx):
    a = os.path.basename(fx)
    #print df.loc[df['filename']==a,'artist']
    name = df.loc[df['filename'] == a,'artist'].iloc[0]
    #print name                                                    #iloc was added so that i get only artist name and not the whole series
    o_h_base = np.zeros(len(artist_names))
    b = artist_names.index(name)
    o_h_base[b]+=1
    o_h_base = o_h_base.reshape([1,9])
    return o_h_base

In [10]:
images = np.concatenate([load_image(f) for f in files],axis=0)
labels = np.concatenate([load_labels(f) for f in files],axis=0)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train,X_val_test,Y_train,Y_val_test = train_test_split(images,labels,test_size = 0.2)
X_val,X_test,Y_val,Y_test = train_test_split(X_val_test,Y_val_test,test_size=0.5)
print 'input(train) shape:', X_train.shape
print 'input(validate)  shape:' , X_val.shape
print 'input(test) shape',X_test.shape
print '----------------------'
print 'output(train) shape:', Y_train.shape
print 'output(validate) shape:',Y_val.shape
print 'output(test) shape:',Y_test.shape

input(train) shape: (1069, 56, 56, 3)
input(validate)  shape: (134, 56, 56, 3)
input(test) shape (134, 56, 56, 3)
----------------------
output(train) shape: (1069, 9)
output(validate) shape: (134, 9)
output(test) shape: (134, 9)


In [13]:
X_train = np.reshape(X_train,[-1,56*56*3])
X_val = np.reshape(X_val,[-1,56*56*3])
X_test = np.reshape(X_test,[-1,56*56*3])
print X_train.shape
print X_val.shape
print X_test.shape

(1069, 9408)
(134, 9408)
(134, 9408)


In [14]:
model = Sequential()                                      
model.add(Dense(128,input_dim =9408)) 
                                                        #basically architecture of the NN
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(Dense(64))
model.add(keras.layers.LeakyReLU(alpha=0.3))
model.add(keras.layers.Dropout(0.3))
model.add(Dense(9))
model.add(Activation('softmax'))

In [15]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               1204352   
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 9)                 585       
__________

In [17]:
model.fit(X_train,Y_train,batch_size=22,epochs=10,validation_data=(X_val,Y_val))

Train on 1069 samples, validate on 134 samples
Epoch 1/10
1069/1069 [==============================] - 3s - loss: 2.2094 - acc: 0.1964 - val_loss: 2.9861 - val_acc: 0.1567
Epoch 2/10
1069/1069 [==============================] - 1s - loss: 1.8028 - acc: 0.3442 - val_loss: 2.4673 - val_acc: 0.2239
Epoch 3/10
1069/1069 [==============================] - 1s - loss: 1.6021 - acc: 0.4387 - val_loss: 2.6112 - val_acc: 0.2239
Epoch 4/10
1069/1069 [==============================] - 1s - loss: 1.3412 - acc: 0.5659 - val_loss: 2.6459 - val_acc: 0.2537
Epoch 5/10
1069/1069 [==============================] - 1s - loss: 1.1359 - acc: 0.6370 - val_loss: 2.7482 - val_acc: 0.3134
Epoch 6/10
1069/1069 [==============================] - 1s - loss: 0.9104 - acc: 0.7362 - val_loss: 2.7028 - val_acc: 0.2985
Epoch 7/10
1069/1069 [==============================] - 1s - loss: 0.7413 - acc: 0.7811 - val_loss: 2.6992 - val_acc: 0.2463
Epoch 8/10
1069/1069 [==============================] - 1s - loss: 0.6093 - ac

In [18]:
model.evaluate(X_test,Y_test,batch_size=22)

 22/134 [===>..........................] - ETA: 0s

[3.7739289945630885, 0.23134328780779198]